# California House Prices with MLP

### $\textbf{Task: Create a MLP regressor for predicting California home prices}$

In [16]:
from typing import Dict, Tuple, List

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import flax.linen as nn
from jax.nn.initializers import lecun_normal, lecun_uniform
import optax
import jax
from jax.nn import one_hot
import jax.numpy as jnp

In [17]:
df = pd.read_csv("datasets/housing.csv")
df.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [18]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [19]:
nan_indexes = df.isna().sum(axis=0)
print(f"Nan element counts:\n{nan_indexes}")
df = df.dropna()

Nan element counts:
longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64


In [20]:
numerical_cols = df.select_dtypes(include=['number']).columns
df['ocean_proximity'], _ = pd.factorize(df['ocean_proximity'])
train_df, test_df = train_test_split(df, test_size = 0.1)

In [21]:
def get_moments(df : pd.DataFrame, cols : List[str]) -> Dict[str, Tuple[float, float]]:
    col_moment_map : Dict[str, Tuple[float, float]] = dict()
    for col in cols:
        col_mean = df.loc[:, col].mean()
        col_std = df.loc[:, col].std(ddof=1)
        col_moment_map[col] = (col_mean, col_std)
    return col_moment_map

def standardise(df : pd.DataFrame, moment_map) -> pd.DataFrame:
    for col in moment_map.keys():
        df[col] = (df[col] - moment_map[col][0]) / moment_map[col][1]
    return df

In [22]:
train_moments = get_moments(train_df, numerical_cols)
train_df = standardise(train_df, train_moments)
test_df = standardise(test_df, train_moments)
train_df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
count,1.838900e+04,1.838900e+04,1.838900e+04,1.838900e+04,1.838900e+04,1.838900e+04,1.838900e+04,1.838900e+04,1.838900e+04,18389.000000
mean,-4.854673e-15,-1.339247e-15,5.583415e-17,-4.675386e-17,7.186957e-17,2.936606e-17,5.023142e-17,-1.275105e-16,7.843829e-17,1.466801
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.853501
min,-2.391925e+00,-1.447872e+00,-2.189491e+00,-1.204508e+00,-1.273583e+00,-1.248638e+00,-1.304326e+00,-1.775834e+00,-1.664761e+00,0.000000
25%,-1.111558e+00,-7.967580e-01,-8.416473e-01,-5.424267e-01,-5.746356e-01,-5.619618e-01,-5.725872e-01,-6.878501e-01,-7.573493e-01,1.000000
50%,5.389158e-01,-6.421769e-01,3.048691e-02,-2.333032e-01,-2.453010e-01,-2.291340e-01,-2.354645e-01,-1.759478e-01,-2.351156e-01,1.000000
75%,7.789846e-01,9.738985e-01,6.647664e-01,2.324369e-01,2.569936e-01,2.622249e-01,2.741395e-01,4.586259e-01,5.022577e-01,2.000000
max,2.629515e+00,2.960032e+00,1.854040e+00,1.674840e+01,1.399428e+01,3.000126e+01,1.458748e+01,5.858319e+00,2.542621e+00,4.000000


In [23]:
test_df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
count,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000,2044.000000
mean,-0.015830,0.010776,0.013963,-0.015722,-0.015678,-0.032068,-0.017426,-0.009111,-0.003322,1.446673
std,1.020705,1.007411,0.983436,0.978027,0.983952,0.922391,0.990902,0.999711,1.014171,0.860621
min,-2.346912,-1.438504,-2.110206,-1.200399,-1.266475,-1.239879,-1.293873,-1.775834,-1.556315,0.000000
25%,-1.137815,-0.796758,-0.762362,-0.552700,-0.581744,-0.556926,-0.583041,-0.697393,-0.759084,1.000000
50%,0.513909,-0.642177,0.030487,-0.246088,-0.250040,-0.246651,-0.252451,-0.181529,-0.247694,1.000000
75%,0.783986,0.983267,0.664766,0.213145,0.241001,0.210549,0.253233,0.457836,0.499872,2.000000
max,2.504480,2.927242,1.854040,11.520240,10.634589,9.220547,11.380894,5.858319,2.542621,3.000000


In [24]:
print(train_df.isna().any())

longitude             False
latitude              False
housing_median_age    False
total_rooms           False
total_bedrooms        False
population            False
households            False
median_income         False
median_house_value    False
ocean_proximity       False
dtype: bool


## Modelling

In [25]:
def dropout(rng, x, rate):
    mask = jax.random.bernoulli(rng, shape=x.shape, p=rate)
    return x * mask / rate

In [26]:
class MLP(nn.Module):
    hidden_layer_sizes : list

    @nn.compact
    def __call__(self, x, rng, rate = 1.0):
        for hidden_dim in self.hidden_layer_sizes[:-1]:
            rng, rng_dropout = jax.random.split(rng, 2)
            x = nn.Dense(features=hidden_dim, kernel_init=lecun_normal())(x)
            x = nn.sigmoid(x)
            x = dropout(rng, x, rate)
        x = nn.Dense(features=self.hidden_layer_sizes[-1], kernel_init = lecun_normal())(x)
        return nn.sigmoid(x)

model = MLP(hidden_layer_sizes=[10, 10, 1])

In [27]:
def get_batch(df, batch_size):
    batch_samples = df.sample(batch_size)
    batch_inputs = batch_samples.loc[:, df.columns != "median_house_value"]
    batch_outputs = batch_samples.loc[:, "median_house_value"]
    encoded_proximities = one_hot(batch_inputs.loc[:, "ocean_proximity"].values, num_classes = 5)
    batch_inputs = batch_inputs.loc[:, batch_inputs.columns != "ocean_proximity"]
    batch_inputs = jnp.concatenate([batch_inputs.values, encoded_proximities], axis=1)
    return jnp.array(batch_inputs), jnp.array(batch_outputs)

In [28]:
rng_key = jax.random.PRNGKey(42)
rng_key, init_rng, dropout_rng = jax.random.split(rng_key, 3)

num_epochs = 1
batch_size = 128
N = train_df.shape[0]

example_batch = jnp.ones((batch_size, 13))
params = model.init(init_rng, example_batch, dropout_rng)

optimiser = optax.chain(
    optax.clip(1.0),
    optax.adam(learning_rate = 5e-3)
)

optimiser_state = optimiser.init(params)

def mse_loss(params, batch_in, batch_out, rng):
    return jnp.mean(jnp.square(model.apply(params, batch_in, rng, rate=0.1) - batch_out))

In [29]:
for epoch in range(num_epochs):
    batch_losses = []
    for batch in range(N // batch_size):
        dropout_rng, batch_rng = jax.random.split(dropout_rng, 2)
        batch_inputs, batch_outputs = get_batch(train_df, batch_size)
        loss, grads = jax.value_and_grad(mse_loss, argnums=(0))(params, batch_inputs, batch_outputs, batch_rng)
        #grads = optax.clip_by_global_norm(grads, 1.0)
        updates, optimiser_state = optimiser.update(grads, optimiser_state)
        params = optax.apply_updates(params, updates)
        batch_losses.append(loss)        
    if ((epoch + 1) % 1 == 0):
        print(f"Epoch {epoch + 1} average training loss: {jnp.array(batch_losses).mean()}")

Epoch 1 average training loss: 1.0709253549575806
